In [91]:
from jupyterscad import view
from  solid2 import * 

In [92]:
# dimensions 
debug_matrix = [0,0,20]
null_matrix = [0,0,0]
debug = True
set_global_fn(40)

base_r = 58
base_d = 20
base_w = 2
gap = 3

inner_r = base_r 
inner_d = base_d
inner_w = base_w 

outer_r = base_r + base_w + gap
outer_d = base_d
outer_w = base_w

In [93]:
inner_base = cylinder(inner_d,inner_r,inner_r) 
inner_diff = cylinder(inner_d,inner_r - inner_w,inner_r - inner_w ) 

inner = inner_base - inner_diff

In [94]:
outer_base = cylinder(outer_d,outer_r+gap,outer_r+gap) 
outer_diff = cylinder(outer_d,outer_r+gap - outer_w,outer_r+gap - outer_w ) 

outer = outer_base - outer_diff

In [98]:
mag_count = 12
mag_x,mag_y,mag_z = [2,6,15]
magnets = []
for magnet in range(0, mag_count):
    magnets += cube([mag_x,mag_y,mag_z]).translate([base_r,0,(base_d-mag_z)/2]).rotate([0,0,magnet*(360/mag_count)])

In [99]:
if debug:
    obj = (inner + magnets).translate(debug_matrix) + outer
else:
    obj = inner + outer + magnets

view(obj,  outfile='body.stl')

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.7, position=(3.0, 5.0,…